<a href="https://colab.research.google.com/github/lyuoveta/git_test_model_1000/blob/master/Llama/Llam_1.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.6 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 6.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 5.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 20.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 15.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 8.3 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-wzz325n8/overlay/local/bin
    changing mode of /tmp/pip-build-env-wzz325n8/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-wzz325n8/overlay/lo

In [ ]:
import re
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:

from google.colab import files


pd= files.upload()

Saving tagged_data_llama_256.csv to tagged_data_llama_256.csv


In [ ]:
import pandas as pd
import random
df = pd.read_csv('tagged_data_llama_256.csv')


In [ ]:
df = df[['reviews', 'Correct review ', 'Incorrect review ']]
df.head()

,reviews,Correct review,Incorrect review
0,Not bad but could be better,1,0
1,This is a test review post done by Amar 4,1,0
2,This is a test review post done by Amar 3,1,0
3,Taking biotin 5000mcg from October first week ...,1,0
4,My nails grew in noticeably thicker and strong...,1,0


In [ ]:
df_incorrect = df.loc[df['Incorrect review '] == 1]['reviews'].reset_index(drop=True)
df_correct = df.loc[df['Correct review '] == 1]['reviews'].reset_index(drop=True)
df_correct[random.choice(range(len(df_correct)))]

'Nebulising Ionic silver was the backbone to curing my discoid eczema. The only side effect is you may never have to visit your doctor again... FACT!'

In [ ]:
df_incorrect[random.choice(range(len(df_incorrect)))]

'I have just started treatment 3 weeks ago and I learned my Vitamin D was way too low.  So, I will see how my body respond.'

In [ ]:
#Step 6: Generating the Response

In [ ]:
for _ in range(20):
    prompt = f'''Please evaluate the following review for correctness. A '1' indicates that the "
            "review is correct (it is not spam and contains information about the product effects), "
            "while a '0' indicates an incorrect review (it is spam or doesn't contain information about the product effects):
    1. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1
    2. {df_incorrect[random.choice(range(len(df_incorrect)))][:256]}
     ANSWER: 0
    3. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1
    4. {df_incorrect[random.choice(range(len(df_incorrect)))][:256]}
     ANSWER: 0
    5. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1

    Please provide the correctness of the following review by entering 0 for incorrect or 1 for correct:
    "{df['reviews'][random.choice(range(len(df)))]}"
    '''
    prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.

    USER: {prompt}

    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.2, top_p=0.95,
                      repeat_penalty=1.2, top_k=150,
                      echo=True)
    print(response)
    print(response["choices"][0]["text"])
    response_text = response["choices"][0]["text"]
    matches = re.findall(r'ASSISTANT:.*?([01])', response_text, re.DOTALL)

    print(matches)



{'id': 'cmpl-5aca1ad2-013a-4948-a6f1-7b6dafc6b58e', 'object': 'text_completion', 'created': 1701359524, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Please evaluate the following review for correctness. A \'1\' indicates that the "\n            "review is correct (it is not spam and contains information about the product effects), "\n            "while a \'0\' indicates an incorrect review (it is spam or doesn\'t contain information about the product effects):\n    1. I feel like Melatonin helps me fall to sleep easier, but I still have trouble staying asleep.\n     ANSWER: 1\n    2. I am taking Purity krill oil to improve brain function, memory, etc.   Nowhere in this article does it list that as a reason to take krill oil.  Please clarify!!   Than

Llama.generate: prefix-match hit


{'id': 'cmpl-cffd75f2-bac6-4225-9d65-2197f4713d66', 'object': 'text_completion', 'created': 1701359592, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Please evaluate the following review for correctness. A \'1\' indicates that the "\n            "review is correct (it is not spam and contains information about the product effects), "\n            "while a \'0\' indicates an incorrect review (it is spam or doesn\'t contain information about the product effects):\n    1. Pretty good treatment...overall I\'d recommend.\n     ANSWER: 1\n    2. Prescribed by my Dr....I am taking 50,000 mg weekly....1 weekly\n     ANSWER: 0\n    3. this relates to aabout four years ago.  I would wake up in the morning with intense calf muscle cramps.  doc prescribed 10,000

Llama.generate: prefix-match hit


{'id': 'cmpl-5a78dd6a-710f-47f1-808b-42b7a2804073', 'object': 'text_completion', 'created': 1701359647, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Please evaluate the following review for correctness. A \'1\' indicates that the "\n            "review is correct (it is not spam and contains information about the product effects), "\n            "while a \'0\' indicates an incorrect review (it is spam or doesn\'t contain information about the product effects):\n    1. I don\'t feel so stiff in the morn.\n     ANSWER: 1\n    2. I been taking 2g of Neptune NOW Krill Oil once a day for three months due to high Cholostrol. i haven\'t retested yet. I will post result after testing.\n     ANSWER: 0\n    3. I have only taken 2 tablets and will not be takin

...

{'id': 'cmpl-abe1aac1-3862-43be-b578-f9454d7fa38a', 'object': 'text_completion', 'created': 1701360565, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Please evaluate the following review for correctness. A \'1\' indicates that the "\n            "review is correct (it is not spam and contains information about the product effects), "\n            "while a \'0\' indicates an incorrect review (it is spam or doesn\'t contain information about the product effects):\n    1. I took very high dose for about 28 days to treat Hepatitis B. The virus count went down from 5.2 million/ml to 3.8 million/ml\n     ANSWER: 1\n    2. Its too soon to that I was prescibed this   folic acid, but I  wholehardly believe in vitatmins.\n     ANSWER: 0\n    3. NOT working on 

Llama.generate: prefix-match hit


{'id': 'cmpl-98b40c18-da6d-445f-ae3d-337deabe7993', 'object': 'text_completion', 'created': 1701360639, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Please evaluate the following review for correctness. A \'1\' indicates that the "\n            "review is correct (it is not spam and contains information about the product effects), "\n            "while a \'0\' indicates an incorrect review (it is spam or doesn\'t contain information about the product effects):\n    1. After being diagnosed with Alzheimer\'s disease over 5 years ago, I took Aricept. Last year, I started taking Ginkgo, and it seems to work as well for me as the Aricept did.\n     ANSWER: 1\n    2. Vitamin D3 - take 2000 a day.\n     ANSWER: 0\n    3. Just started  d3 andastraight away

Llama.generate: prefix-match hit


{'id': 'cmpl-8349734e-b129-40b9-a433-9ddedf045b0c', 'object': 'text_completion', 'created': 1701360685, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Please evaluate the following review for correctness. A \'1\' indicates that the "\n            "review is correct (it is not spam and contains information about the product effects), "\n            "while a \'0\' indicates an incorrect review (it is spam or doesn\'t contain information about the product effects):\n    1. This is for Rob and anyone else who doesn\'t know about the D Vitamins.    D3 is the one you want to purchase and use, over D2. Our bodies convert D2 to D3. So, just get the D3, and you will have better results.\n     ANSWER: 1\n    2. I have hemolytic anemia and was advised by my doct

In [1]:
!pip install scikit-learn

true_labels = [1, 0, 0, 0, 1,1,0,1,0,1,1]
predicted_labels = [0,0,0,1,1,1,0,0,1,0,1]

correct_predictions = sum(1 for true, predicted in zip(true_labels, predicted_labels) if true == predicted)

accuracy = correct_predictions / len(true_labels)

print(f"Точность модели: {accuracy * 100:.2f}%")


Точность модели: 54.55%


In [2]:
true_labels = [1, 0, 0, 0, 1,1,0,1,0,1,1,1,0,1,1,1]
predicted_labels = [0,0,0,1,1,1,0,0,1,0,1,1,0,0,0,1]

correct_predictions = sum(1 for true, predicted in zip(true_labels, predicted_labels) if true == predicted)

accuracy = correct_predictions / len(true_labels)

print(f"Точность модели: {accuracy * 100:.2f}%")

Точность модели: 56.25%
